# **Jupyter notebook for time series forecasting based on machine learning**
### **01 Description**

This Jupyter notebook is used for traffic flow time series prediction, using the GRU model and the TGCN model (https://ieeexplore.ieee.org/abstract/document/8809901). The experimental data comes from the following related code.

### **02 Import libraries and complete instantiation**

In [1]:
# Import necessary modules
from utils.data_process import create_dataloader
from instance_parameters import Predict
from models.exp2_gru import GRU
from models.exp2_seggru import SEGGRU
from utils.predict import predict
from utils.test_and_inspect_fit import test, inspect_model_fit
from utils.train import train
import torch

**The parameters you can adjust** (for example, instance = Predict().model = 'GRU')
**Parameter**     | **Value**
-------- | -----
model  | 'TGCN'/'GRU'
input_size  | 40
use_gpu  | False
device  | 0
lr  | 0.001
epochs  | 100
batch_size  | 32
hidden_size  | 64
data_path  | 'dataset/filled_sz_speed.csv'
target  | '92878'
window_size  |  48
pre_len  | 4
shuffle  | True

In [2]:

# Create an instance of the Predict class
instance = Predict()
instance.epochs = 20
instance.model ='GRU'
instance.pre_len = 4
instance.input_size = 5
instance.batch = 4
instance.hidden_size = 10
instance.data_path = 'dataset/nba.csv'
instance.target = 'Suns'
instance.window_size = 16
instance.pre_len = 1

### **03 Confirm used device（GPU or CPU）**

In [3]:
if isinstance(instance.device, int) and instance.use_gpu:
    device = torch.device("cuda:" + f'{instance.device}')
else:
    device = torch.device("cpu")
train_loader, test_loader, valid_loader, scaler = create_dataloader(instance, device)
print("Used device:", device)

>>>>>>>>>>>>>>>>>>>>>>>>>>>>Creating DataLoader<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
Training set size: 170 Test set size: 49 Validation set size: 25
Training set data:    154 Converted to batch data: 6
Test set data:        33 Converted to batch data: 1
Validation set data:  9 Converted to batch data: 0
>>>>>>>>>>>>>>>>>>>>>>>>>>>>DataLoader Created<<<<<<<<<<<<<<<<<<<<<<<<<<<
Used device: cpu


### 03 Instantiate the model

In [4]:
try:
    print(f"Start initializing the {instance.model} model")
    if instance.model == 'GRU':
        model = GRU(instance, device).to(device)
    elif instance.model == 'SEGGRU':
        model = SEGGRU(instance, device).to(device)
    print(f"Initialization of the {instance.model} model is complete")
except Exception as e:
    print(f"Initialization of the {instance.model} model failed")
    print(f"Failure Details:{e}")

Start initializing the GRU model
Initialization of the GRU model is complete


### 04 Train model

In [5]:
if instance.train:
    print(f"Start {instance.model} model training")
    train(model, instance, train_loader, scaler)

Start GRU model training


100%|██████████| 20/20 [00:02<00:00,  7.17it/s]


>>>>>>>>>>>>>>>>>>>>>>模型已保存,用时:0.0519 min<<<<<<<<<<<<<<<<<<


### 05 test model

In [6]:
if instance.test:
    print(f">>>>>>>>>>>>>>>>>>>>>>>>>开始{instance.model}模型测试<<<<<<<<<<<<<<<<<<<<<<<<<<<")
    test(model, instance, test_loader, scaler)

>>>>>>>>>>>>>>>>>>>>>>>>>开始GRU模型测试<<<<<<<<<<<<<<<<<<<<<<<<<<<
Testset Mean Absolute Error(测试集平均绝对误差): 0.84240204
Testset Root Mean Squared Error(测试集均方根误差): 1.1510292


Target Feature Testset R2(目标特征测试集拟合曲线决定系数): -0.920701351418272


### 06 Inspect model fitting results

In [7]:
if instance.inspect_fit:
    print(f">>>>>>>>>>>>>>>>>>>>>>>>>开始检验{instance.model}模型拟合情况<<<<<<<<<<<<<<<<<<<<<<<<<<<")
    inspect_model_fit(model, instance, train_loader, scaler)

>>>>>>>>>>>>>>>>>>>>>>>>>开始检验GRU模型拟合情况<<<<<<<<<<<<<<<<<<<<<<<<<<<


Target Feature Trainingset R2(目标特征训练集拟合曲线决定系数): 0.1516775076441116


### 07 Predict and draw the result

In [8]:
if instance.predict:
    print(f">>>>>>>>>>>>>>>>>>>>>>>>>预测未来{instance.pre_len}条数据<<<<<<<<<<<<<<<<<<<<<<<<<<<")
    predict(model, instance, device, scaler)

>>>>>>>>>>>>>>>>>>>>>>>>>预测未来1条数据<<<<<<<<<<<<<<<<<<<<<<<<<<<


IndexError: tuple index out of range